In [1]:
import { load } from "dotenv";
const env = await load();

const process = { env };
process.env;


{
  OPENAI_API_KEY: "sk-b72b74abd3f541a08b68dc756de84958",
  MODEL_NAME: "qwen-turbo-1101",
  BASE_URL: "https://dashscope.aliyuncs.com/compatible-mode/v1",
  DEEPSEEK_API_KEY: "sk-b1a4cf12df35436f972ba72d4e7c669b",
  EMBEDDING_MODEL_NAME: "text-embedding-v1",
  AZURE_OPENAI_API_KEY: "923274d2ec144d95b33adad2c29a362e",
  AZURE_OPENAI_API_INSTANCE_NAME: "gemdalechatgpt4",
  AZURE_OPENAI_API_VERSION: "2023-07-01-preview",
  AZURE_OPENAI_API_DEPLOYMENT_NAME: "jdkjkjgpt4turbo",
  AZURE_EMBEDDING_MODEL_NAME: "text-embedding-ada-002",
  SERP_API_KEY: "185032b32dc536d633129d221f7f7be48629f0e4569138241d817449166e21ca"
}

In [2]:
import { TextLoader } from "langchain/document_loaders/fs/text";
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
import { MemoryVectorStore } from "langchain/vectorstores/memory";
import { OpenAIEmbeddings } from "@langchain/openai";

// 加载知识库
const loader = new TextLoader("documents/chatbot.txt");
const docs = await loader.load();
// 切分（超出LLM上下文限制）
const splitter = new RecursiveCharacterTextSplitter({
  chunkSize: 100,
  chunkOverlap: 20,
});

const splitDocs = await splitter.splitDocuments(docs);

// 构建 vector store
const embeddings = new OpenAIEmbeddings({
  model: process.env.EMBEDDING_MODEL_NAME,
  batchSize: 20,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});
// const embeddings = new OpenAIEmbeddings({
//   configuration: {
//     baseURL: `https://${process.env.AZURE_OPENAI_API_INSTANCE_NAME}.openai.azure.com/openai/deployments/${process.env.AZURE_EMBEDDING_MODEL_NAME}`,
//     apiKey: process.env.AZURE_OPENAI_API_KEY,
//     defaultQuery: {
//       "api-version": process.env.AZURE_OPENAI_API_VERSION,
//     },
//   },
// });

const vectorstore = new MemoryVectorStore(embeddings);
await vectorstore.addDocuments(splitDocs);


In [3]:
import { RunnableSequence } from "@langchain/core/runnables";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { ChatOpenAI } from "@langchain/openai";
import { JSONChatHistory } from "./node/history/index.ts";
import { BufferMemory } from "langchain/memory";
import { ConversationChain } from "langchain/chains";

// // 构建retriever
const retriever = vectorstore.asRetriever(2);
// const res = await retriever.invoke("原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论")

// console.log(res)

// 将获取到的关联上下文处理成纯文字
const convertDocsToString = (documents: Document[]): string => {
  return documents.map((document) => document.pageContent).join("\n");
};

const contextRetrieverChain = RunnableSequence.from([
  (input) => input.question,
  retriever,
  convertDocsToString,
]);

// await contextRetriverChain.invoke({
//   question: "球形闪电",
// });

// 构建prompt
const TEMPLATE = `
你是一个熟知内部知识库的机器人，你在回答时会引用知识库，并擅长通过自己的总结归纳，组织语言给出答案。
并且回答时仅根据知识库，尽可能回答用户问题，如果知识库中没有相关内容，你可以回答“原文中没有相关内容”，不要回答知识库以外的内容。

以下是知识库中跟用户回答相关的内容：
{context}

现在，你需要基于知识库，回答以下问题：
{question}`;

const prompt = ChatPromptTemplate.fromTemplate(TEMPLATE);

const llm = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});
// const llm = new ChatOpenAI({
//   configuration: {
//     baseURL: `https://${process.env.AZURE_OPENAI_API_INSTANCE_NAME}.openai.azure.com/openai/deployments/${process.env.AZURE_OPENAI_API_DEPLOYMENT_NAME}`,
//     apiKey: process.env.AZURE_OPENAI_API_KEY,
//     defaultQuery: {
//       "api-version": process.env.AZURE_OPENAI_API_VERSION,
//     },
//   },
// });

// function getContext(question: string) {
//   console.log(question)
//   return '球形闪电是一种面包'
// }


const history = new JSONChatHistory({
  dir: "chat_data",
  sessionId: "chatbot",
})

const memory = new BufferMemory({ chatHistory: history });

const ragChain = RunnableSequence.from([
  // RunnableParallel.from({
  //     context: contextRetrieverChain,
  //     question: (input) => input.question,
  // }),
  // async (input) => ({
  //   context: await contextRetrieverChain.invoke({ question: input.question }),
  //   question: input.question
  // }),
  {
    context: async (input) => {
      const context = await contextRetrieverChain.invoke({ question: input.question });
      console.log('context------', context)
      return context
    },
    // context: contextRetrieverChain,
    question: (input) => input.question,
  },
  prompt,
  llm,
  new StringOutputParser(),
]);

// const chain = new ConversationChain({ llm: ragChain, memory });


In [4]:
await ragChain.invoke({
  question: "叶文洁是什么人",
});

context------ 洁，像新的一样，脸色有些苍白，但目光清澈而平静。她最后在父亲面前站住了。
我们认识，她就是许多年前在深夜的大学图书馆里说我很有目的性并问我在找什么的那位漂亮女生，我想了半天才想起她的名字：戴琳。


"叶文洁是脸色有些苍白，但目光清澈而平静的一个人物。她是许多年前在深夜的大学图书馆里与“我”有过交谈的那位女性，被描述为漂亮的女生，并且曾经对“我”说很有目的性并询问“我”在找什么。后来，“我”想起了她的名字叫戴琳。因此，叶文洁就是指戴琳。请注意，知识库中并没有直接提到“叶文洁”这个名字，而是通过描述和后续的信息关联到了戴琳。"